In [1]:
import datetime
import json
import requests
import pprint
from poseidon import poseidon
from urllib.request import pathname2url
from hashlib import sha256
import hashlib
import calendar
import time
#import commond.ipynb from same folder
import import_ipynb
from conf_ppe import *
from common import *

importing Jupyter notebook from conf_ppe.ipynb
importing Jupyter notebook from common.ipynb
03 Aug 2020 DAQ FilesPQC887


In [2]:
def download_file(address, orgId, file_path, assetId):
    try:
        urlAsset = '%s/connect-service/v2.1/files?action=download&orgId=%s&fileUri=%s&assetId=%s&category=feature' % (address, orgId, file_path, assetId)
        print(urlAsset)
        req = poseidon.urlopen(appkey, appsecret, urlAsset)
        return req;
    except Exception as err:
        print(err)   

In [3]:
def encrypt_string(hash_string):
    sha_signature = hashlib.sha256(hash_string.encode()).hexdigest()
    return sha_signature

In [4]:
def get_access_token(address, body):
    try:
        custom_header = {'content-type': 'application/json'}
        urlAsset = '%s/apim-token-service/v2.0/token/get' % (address)
        print(urlAsset)
        req = requests.post(urlAsset, data = json.dumps(body), headers = custom_header)
        output = req.json()
        result = output['data']['accessToken']
        return result
    except Exception as err:
        print(err)   

In [5]:
def getTokenRequestBody():
    try:
        timestamp = int(round(time.time() * 1000))
        unixtime = timestamp
        sign = '{}{}{}'.format(appkey, unixtime, appsecret).lower()
        print(sign)
        encrypted_code = encrypt_string(sign)

        payload = {
            "appKey": appkey,
            "encryption": encrypted_code,
            "timestamp": unixtime
        }
        print(payload)
        return payload
    except Exception as err:
        print(err) 

In [6]:
requestBody = getTokenRequestBody()
print('address: %s' % (address))
parsed = get_access_token(address, requestBody)

print(parsed)

3dc2597b-e232-45db-bb16-e94443deb7f31615469761038d1768b8c-4be1-4b8a-8315-3f370f6228a5
{'appKey': '3dc2597b-e232-45db-bb16-e94443deb7f3', 'encryption': '1ff650ca12b6ed3740b598bd45b5ffb36a72ff1937775c1e7afd40c3154a3b5b', 'timestamp': 1615469761038}
address: https://apim-ppe1.envisioniot.com
https://apim-ppe1.envisioniot.com/apim-token-service/v2.0/token/get
99ab7cb07f198973304f14ede41784e9c52f4de877ceb56b5ce5e0904045665d


In [17]:
def upload_mp(token_API_address, integration_address, org_id, asset_id):
    try:
        requestBody = getTokenRequestBody()
        token = get_access_token(token_API_address, requestBody)
                     
        custom_header = {'content-type': 'application/json',
            'apim-accesstoken': token
        }
        print('header = %s' % custom_header)
        body  = {
            "method": "integration.measurepoint.post",
            "version": "1.1",
            "params":[        
                        {
                            "assetId": "tR7u5in2",
                            "time": int(round(time.time() * 1000)),
                            "measurepoints": {
                                "demand_max_ytd": 12.09000012
                            }
                        }
                    ]
            }
        print('body = %s' % json.dumps(body))
        urlAsset = '%s/connect-service/v2.1/integration?action=postMeasurepoint&orgId=%s' % (integration_address, org_id)
        print('Upload Measurement Point: %s' % urlAsset)
        response = requests.post(urlAsset, data = json.dumps(body), headers = custom_header)
        print('output: %s' % response)
        return response             
    except Exception as err:
        print(err)

In [8]:
parsed = upload_mp(address, integration_address, orgId, assetId)

print(parsed)

3dc2597b-e232-45db-bb16-e94443deb7f31615260616685d1768b8c-4be1-4b8a-8315-3f370f6228a5
{'appKey': '3dc2597b-e232-45db-bb16-e94443deb7f3', 'encryption': 'b2c7d7bfd4a6e9496a4155dcf02136260c39a8b693d5d5222ee55037db297a21', 'timestamp': 1615260616685}
https://apim-ppe1.envisioniot.com/apim-token-service/v2.0/token/get
header = {'content-type': 'application/json', 'apim-accesstoken': 'ccd112edb9b19bb99f79581d66a0c2be391fecbbc651cba4ca6e5c9db0306973'}
body = {"method": "integration.measurepoint.post", "version": "1.1", "params": [{"assetId": "tR7u5in2", "time": 1615260616767, "measurepoints": {"demand_max_ytd": 12.09000012}}]}
Upload Measurement Point: https://iot-http-integration-ppe1.envisioniot.com/connect-service/v2.1/integration?action=postMeasurepoint&orgId=o15897810838641
output: <Response [403]>
<Response [403]>
